## Creating Partitioned Tables

We can also create partitioned tables as part of Spark Metastore Tables.

* There are some challenges in creating partitioned tables directly using `spark.catalog.createTable`.
* But if the directories are similar to partitioned tables with data, we should be able to create partitioned tables.
* Let us create partitioned table for `orders` by `order_month`.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [ ]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Spark Metastore'). \
    master('yarn'). \
    getOrCreate()

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

In [ ]:
spark.conf.set('spark.sql.shuffle.partitions', '2')

### Tasks

Let us perform tasks related to partitioned tables.
* Read data from file into data frame.
* Add additional column which will be used to partition the data.
* Write the data into the target location on which we are going to create the table.
* Create partitioned table using the location to which we have copied the data and validate.
* We can recover partitions by running `MSCK REPAIR TABLE` using `spark.sql` or by invoking `spark.catalog.recoverPartitions`.
* When we use `createTable` to create partitioned table, we have to recover partitions so that partitions are visible.

In [ ]:
import getpass
username = getpass.getuser()

In [ ]:
spark.sql(f'CREATE DATABASE IF NOT EXISTS {username}_retail')

In [ ]:
spark.catalog.setCurrentDatabase(f'{username}_retail')

In [ ]:
spark.catalog.currentDatabase()

In [ ]:
orders_path = '/public/retail_db/orders'

In [ ]:
%%sh

hdfs dfs -ls /public/retail_db/orders

In [ ]:
spark.sql('DROP TABLE IF EXISTS orders_part')

In [ ]:
%%sh

hdfs dfs -ls /user/`whoami`/retail_db/orders_part

In [ ]:
%%sh

hdfs dfs -rm -R -skipTrash /user/`whoami`/retail_db/orders_part

In [ ]:
from pyspark.sql.functions import date_format

In [ ]:
spark. \
    read. \
    csv(orders_path,
        schema='''order_id INT, order_date DATE,
                  order_customer_id INT, order_status STRING
               '''
       ). \
    withColumn('order_month', date_format('order_date', 'yyyyMM')). \
    write. \
    partitionBy('order_month'). \
    parquet(f'/user/{username}/retail_db/orders_part')

In [ ]:
%%sh

hdfs dfs -ls /user/`whoami`/retail_db/orders_part

In [ ]:
%%sh

hdfs dfs -ls -R /user/`whoami`/retail_db/orders_part

In [ ]:
spark. \
    read. \
    parquet(f'/user/{username}/retail_db/orders_part/order_month=201308'). \
    show()

In [ ]:
spark. \
    read. \
    parquet(f'/user/{username}/retail_db/orders_part'). \
    show()

In [ ]:
spark. \
    catalog. \
    createTable('orders_part',
                path=f'/user/{username}/retail_db/orders_part',
                source='parquet'
               )

In [ ]:
spark.read.table('orders_part').show()

In [ ]:
spark.sql('SHOW PARTITIONS orders_part').show()

In [ ]:
spark.catalog.recoverPartitions('orders_part')

In [ ]:
spark.sql('SHOW PARTITIONS orders_part').show()

In [ ]:
spark.read.table('orders_part').show()

In [ ]:
spark.sql('SELECT order_month, count(1) FROM orders_part GROUP BY order_month').show()

In [ ]:
spark.read.table('orders_part'). \
    groupBy('order_month'). \
    count(). \
    show()